In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator
from sklearn.preprocessing import StandardScaler

In [3]:
import logging

logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)

In [4]:
data = np.load("./nsi_data/sample_nsi_regression_1e7_v1.npz")

In [5]:
data_selection = 1 # 0 for all, 1 for lowE(<5GeV), 2 for high(>5GeV)

if data_selection == 0:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif data_selection == 1:
    data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])
elif data_selection == 2:
    data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])

pre_target = np.column_stack([data['theta13'],data['theta23']/5,data['delta']/180+5,data['mumu']+5,data['emu']+5,data['etau']+5])
target = np.column_stack([pre_target, pre_target])

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:1000000]
y_test = target[900000:1000000]

In [6]:
model_name = "nsi_degen_allparams"

In [10]:
from tensorflow.keras import backend as K

def custom_mse(y_true, y_pred):
    loss = K.square(y_pred - y_true)
    loss1 = loss[0]+loss[1]+loss[2]+loss[3]+loss[4]+loss[5]
    loss2 = loss[6]+loss[7]+loss[8]+loss[9]+loss[10]+loss[11]
    return K.switch(loss1<loss2, loss1, loss2)


def DNN_Model(name):
    model_DNN = Sequential(name = "Model_DNN_"+str(name))

    model_DNN.add(keras.Input(shape=(len(x_train[0]),), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_2'))

    model_DNN.add(Dense(12, activation='relu', name = 'output'))
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss=custom_mse,#keras.losses.binary_crossentropy
                              optimizer=model_opt,
                              metrics=['mse'])
    model_DNN.summary()
    return model_DNN

In [11]:
model = DNN_Model("regression")

Model: "Model_DNN_regression"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BatchNormalization (BatchNor (None, 144)               576       
_________________________________________________________________
dense_1 (Dense)              (None, 512)               74240     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
output (Dense)               (None, 12)                6156      
Total params: 343,628
Trainable params: 343,340
Non-trainable params: 288
_________________________________________________________________


In [12]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 37s 3ms/step - loss: 3.3918 - mse: 0.7693 - val_loss: 0.5996 - val_mse: 0.1592
Epoch 2/20
12516/12516 [==============================] - 35s 3ms/step - loss: 0.5944 - mse: 0.1588 - val_loss: 0.3766 - val_mse: 0.1101
Epoch 3/20
12516/12516 [==============================] - 35s 3ms/step - loss: 0.4426 - mse: 0.1278 - val_loss: 0.3247 - val_mse: 0.1009
Epoch 4/20
12516/12516 [==============================] - 36s 3ms/step - loss: 0.3788 - mse: 0.1135 - val_loss: 0.2801 - val_mse: 0.0867
Epoch 5/20
12516/12516 [==============================] - 36s 3ms/step - loss: 0.3395 - mse: 0.1043 - val_loss: 0.2338 - val_mse: 0.0734
Epoch 6/20
12516/12516 [==============================] - 36s 3ms/step - loss: 0.3171 - mse: 0.0975 - val_loss: 0.2484 - val_mse: 0.0797
Epoch 7/20
12516/12516 [==============================] - 35s 3ms/step - loss: 0.2928 - mse: 0.0929 - val_loss: 0.2194 - val_mse: 0.0682
Epoch 8/20
12516/12516 [=================

In [13]:
model_index = 1
while os.path.isfile('./models/{}_{}.h5'.format(model_name, model_index)):
    model_index += 1
model.save('./models/{}_{}.h5'.format(model_name, model_index))

In [ ]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )
    
    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

3125/3125 [==============================] - 5s 2ms/step - loss: 0.1618 - mse: 0.0518
Epoch 1/5
12516/12516 [==============================] - 35s 3ms/step - loss: 0.2064 - mse: 0.0694 - val_loss: 0.1570 - val_mse: 0.0516
Epoch 2/5
12516/12516 [==============================] - 35s 3ms/step - loss: 0.2052 - mse: 0.0684 - val_loss: 0.1440 - val_mse: 0.0473
Epoch 3/5
12516/12516 [==============================] - 35s 3ms/step - loss: 0.2026 - mse: 0.0677 - val_loss: 0.1486 - val_mse: 0.0500
Epoch 4/5
12516/12516 [==============================] - 35s 3ms/step - loss: 0.2014 - mse: 0.0675 - val_loss: 0.1417 - val_mse: 0.0462
Epoch 5/5
3125/3125 [==============================] - 5s 2ms/step - loss: 0.1371 - mse: 0.0461
Epoch 1/5
12516/12516 [==============================] - 36s 3ms/step - loss: 0.1976 - mse: 0.0665 - val_loss: 0.1384 - val_mse: 0.0482
Epoch 2/5
12516/12516 [==============================] - 36s 3ms/step - loss: 0.1950 - mse: 0.0655 - val_loss: 0.1551 - val_mse: 0.0542
Ep

In [ ]:
furthur_index = 1
path = './models_furthurTrain/{}_{}_{}.h5'
while os.path.isfile(path.format(model_name, model_index, furthur_index)):
    index += 1
model.save(path.format(model_name, model_index, furthur_index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = './models_furthurTrain/{}_{}_{}_result.npy'.format(model_name, model_index, furthur_index),
        arr = outfile)

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(10):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
index = 1
path = './models_PoissonTrain/{}_{}_{}_{}.h5'
while os.path.isfile(path.format(model_name, model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_name, model_index, furthur_index, index))